In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
trajectories_train_df = pd.read_csv('../data/original/trajectories(table 5)_training.csv',parse_dates=['starting_time'])

In [3]:
trajectories_test_df = pd.read_csv('../data/test/original/trajectories(table 5)_test1.csv',parse_dates=['starting_time'])

In [4]:
trajectories_df = pd.concat([trajectories_train_df, trajectories_test_df])
trajectories_df = trajectories_df.reset_index(drop=True)

In [5]:
link_df = pd.read_csv('../data/original/links (table 3).csv', index_col=0)

In [6]:
def parseSingleTravelSeq(i, s):
    nodes = []
    seq = s['travel_seq'].split(';');
    
    for k, n in enumerate(seq):
        
        info = n.split('#')
        link_id = int(info[0])
        start_time = datetime.datetime.strptime(info[1], '%Y-%m-%d %H:%M:%S')
        consume = float(info[2])
    
        end_time = datetime.datetime.strptime(info[1], '%Y-%m-%d %H:%M:%S') + datetime.timedelta(seconds=consume)
        node = {}
        node['start_time'] = start_time
        node['end_time'] = end_time
        node['link'] = link_id
        node['consume'] = consume
        node['index'] = i
        node['length'] = link_df.loc[link_id]['length']
        nodes.append(node)
        
    return nodes

In [7]:
single_nodes = []
for i in trajectories_df.index.values:
    if i%2000 == 0:
        print(i)
    single_nodes.extend(parseSingleTravelSeq(i, trajectories_df.loc[i]))

0
2000
4000
6000
8000
10000
12000
14000
16000
18000
20000
22000
24000
26000
28000
30000
32000
34000
36000
38000
40000
42000
44000
46000
48000
50000
52000
54000
56000
58000
60000
62000
64000
66000
68000
70000
72000
74000
76000
78000
80000
82000
84000
86000
88000
90000
92000
94000
96000
98000
100000
102000
104000
106000
108000
110000


In [8]:
single_travel_link_df = pd.DataFrame(single_nodes)

In [9]:
twenty_timedelta = datetime.timedelta(minutes=20)
forty_timedelta = datetime.timedelta(minutes=40)
def normlize_start_time(start_time):
    minute = start_time.minute
    second = start_time.second
    
    current_timedelta = datetime.timedelta(minutes=minute, seconds=second)
    
    if current_timedelta < twenty_timedelta:
        return start_time - current_timedelta
    elif current_timedelta < forty_timedelta:
        return start_time - current_timedelta + twenty_timedelta
    else:
        return start_time - current_timedelta + forty_timedelta

In [10]:
def day_start_time(start_time):
    
    normlize_date = normlize_start_time(start_time)
    
    hour = normlize_date.hour
    minute = normlize_date.minute
    second = normlize_date.second
    
    return datetime.time(hour, minute, second)
        
    

In [11]:
def day_start_date(start_time):
    return start_time.date()

In [12]:
single_travel_link_df['norm_start_time'] = single_travel_link_df['start_time'].apply(normlize_start_time)

In [13]:
single_travel_link_df['day_start_time'] = single_travel_link_df['start_time'].apply(day_start_time)

In [14]:
single_travel_link_df['day_start_date'] = single_travel_link_df['start_time'].apply(day_start_date)

# 取每天平均20分钟内的车速情况进行离散值处理

In [15]:
groupby = single_travel_link_df.groupby(['link', 'norm_start_time'])

Q1 = groupby['consume'].quantile(0.25)
Q3 = groupby['consume'].quantile(0.75)
IQR = Q3 - Q1

Q_TOP_LIMIT = Q3 + 1.5 * IQR
Q_BOTTOM_LIMIT = Q1 - 1.5 * IQR

Q_TOP_LIMIT = Q_TOP_LIMIT.reset_index()
Q_BOTTOM_LIMIT = Q_BOTTOM_LIMIT.reset_index()

top_pivot_df = pd.DataFrame.pivot_table(Q_TOP_LIMIT, values='consume', index=['link'], columns=['norm_start_time'])
bottom_pivot_df = pd.DataFrame.pivot_table(Q_BOTTOM_LIMIT, values='consume', index=['link'], columns=['norm_start_time'])

def DiscreteFilter(s):
    
    if (s.name % 10000) == 0:
        print(s.name)
    
    top_consume = top_pivot_df[s['norm_start_time']][s['link']]
    bottom_consume = bottom_pivot_df[s['norm_start_time']][s['link']]

    if s['consume'] > top_consume:
        return 'Top'
    elif s['consume'] < bottom_consume:
        return 'Bottom'
    else:
        return 'Normal'
    
single_travel_link_df['period_discrete'] = single_travel_link_df.apply(DiscreteFilter, axis=1)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000


In [16]:
single_travel_link_df

,consume,end_time,index,length,link,start_time,norm_start_time,day_start_time,day_start_date,current_discrete
0,9.56,2016-07-19 00:14:33.560,0,78,105,2016-07-19 00:14:24,2016-07-19 00:00:00,00:00:00,2016-07-19,Normal
1,6.75,2016-07-19 00:14:40.750,0,58,100,2016-07-19 00:14:34,2016-07-19 00:00:00,00:00:00,2016-07-19,Normal
2,13.00,2016-07-19 00:14:54.000,0,121,111,2016-07-19 00:14:41,2016-07-19 00:00:00,00:00:00,2016-07-19,Normal
3,7.47,2016-07-19 00:15:01.470,0,23,103,2016-07-19 00:14:54,2016-07-19 00:00:00,00:00:00,2016-07-19,Normal
4,32.85,2016-07-19 00:15:34.850,0,197,122,2016-07-19 00:15:02,2016-07-19 00:00:00,00:00:00,2016-07-19,Normal
5,11.58,2016-07-19 00:36:07.580,1,78,105,2016-07-19 00:35:56,2016-07-19 00:20:00,00:20:00,2016-07-19,Normal
6,7.44,2016-07-19 00:36:15.440,1,58,100,2016-07-19 00:36:08,2016-07-19 00:20:00,00:20:00,2016-07-19,Normal
7,16.23,2016-07-19 00:36:31.230,1,121,111,2016-07-19 00:36:15,2016-07-19 00:20:00,00:20:00,2016-07-19,Normal
8,5.95,2016-07-19 00:36:37.950,1,23,103,2016-07-19 00:36:32,2016-07-19 00:20:00,00:20:00,2016-07-19,Normal
9,104.79,2016-07-19 00:38:24.790,1,197,122,2016-07-19 00:36:40,2016-07-19 00:20:00,00:20:00,2016-07-19,Normal


# 用取当天24小时内的车速情况进行离散值分离

In [19]:
groupby = single_travel_link_df.groupby(['link', 'day_start_date'])

Q1 = groupby['consume'].quantile(0.25)
Q3 = groupby['consume'].quantile(0.75)
IQR = Q3 - Q1

Q_TOP_LIMIT = Q3 + 1.5 * IQR
Q_BOTTOM_LIMIT = Q1 - 1.5 * IQR

Q_TOP_LIMIT = Q_TOP_LIMIT.reset_index()
Q_BOTTOM_LIMIT = Q_BOTTOM_LIMIT.reset_index()

top_pivot_df = pd.DataFrame.pivot_table(Q_TOP_LIMIT, values='consume', index=['link'], columns=['day_start_date'])
bottom_pivot_df = pd.DataFrame.pivot_table(Q_BOTTOM_LIMIT, values='consume', index=['link'], columns=['day_start_date'])

def DiscreteFilter(s):
    
    if (s.name % 10000) == 0:
        print(s.name)
    
    top_consume = top_pivot_df[s['day_start_date']][s['link']]
    bottom_consume = bottom_pivot_df[s['day_start_date']][s['link']]

    if s['consume'] >= top_consume:
        return 'Top'
    elif s['consume'] <= bottom_consume:
        return 'Bottom'
    else:
        return 'Normal'
    
single_travel_link_df['day_discrete'] = single_travel_link_df.apply(DiscreteFilter, axis=1)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000


# 取所有天平均20分钟内的车速情况进行离散值分离?

In [16]:
groupby = single_travel_link_df.groupby(['link', 'day_start_time'])

Q1 = groupby['consume'].quantile(0.25)
Q3 = groupby['consume'].quantile(0.75)
IQR = Q3 - Q1

Q_TOP_LIMIT = Q3 + 1.5 * IQR
Q_BOTTOM_LIMIT = Q1 - 1.5 * IQR

Q_TOP_LIMIT = Q_TOP_LIMIT.reset_index()
Q_BOTTOM_LIMIT = Q_BOTTOM_LIMIT.reset_index()

top_pivot_df = pd.DataFrame.pivot_table(Q_TOP_LIMIT, values='consume', index=['link'], columns=['day_start_time'])
bottom_pivot_df = pd.DataFrame.pivot_table(Q_BOTTOM_LIMIT, values='consume', index=['link'], columns=['day_start_time'])

def DiscreteFilter(s):
    
    if (s.name % 10000) == 0:
        print(s.name)
    
    top_consume = top_pivot_df[s['day_start_time']][s['link']]
    bottom_consume = bottom_pivot_df[s['day_start_time']][s['link']]

    if s['consume'] >= top_consume:
        return 'Top'
    elif s['consume'] <= bottom_consume:
        return 'Bottom'
    else:
        return 'Normal'
    
single_travel_link_df['all_discrete'] = single_travel_link_df.apply(DiscreteFilter, axis=1)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
570000
580000
590000
600000
610000
620000
630000
640000
650000
660000
670000
680000
690000
700000
710000
720000
730000
740000
750000
760000
770000
780000


In [20]:
top_discrete_point_df = single_travel_link_df[(single_travel_link_df['day_discrete'] == 'Top') | (single_travel_link_df['current_discrete'] == 'Top')]
bottom_discrete_point_df = single_travel_link_df[(single_travel_link_df['day_discrete'] == 'Bottom') | (single_travel_link_df['current_discrete'] == 'Bottom')]
middle_normal_df = single_travel_link_df[(single_travel_link_df['day_discrete'] == 'Normal') & (single_travel_link_df['current_discrete'] == 'Normal')]

In [21]:
top_discrete_point_df.drop(['day_discrete', 'current_discrete'], axis=1).to_csv('../data/preprocess/top_discrete_point_link.csv', index=False)

In [22]:
bottom_discrete_point_df.drop(['day_discrete', 'current_discrete'], axis=1).to_csv('../data/preprocess/bottom_discrete_point_link.csv', index=False)

In [23]:
middle_normal_df.drop(['day_discrete', 'current_discrete'], axis=1).to_csv('../data/preprocess/middle_normal_link.csv', index=False)

In [24]:
single_travel_link_df.drop(['day_discrete', 'current_discrete'], axis=1).to_csv('../data/preprocess/all_travel_link.csv', index=False)